In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import squarify  
import seaborn as sns
sns.set();
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Метод $k$ ближайших соседей (kNN)

В этой части работы вы:
- научитесь готовить данные к построению модели (предобработка, или препроцессинг данных);
- познакомитесь с методами ближайших соседей для задач классификации, реализованными в библиотеке scikit-learn ;
- научитесь оценивать качество модели с помощью отложенной выборки.

## 1. Подключитесь к одному из предложенных наборов данных:
- Adult income;
- Telecom churn;
- Cardiovascular disease

или выберите свой. Разберитесь в том, как устроен ваш датасет и какова постановка задачи.

In [ ]:
MAIN_PATH = '../input/'
df = pd.read_csv(MAIN_PATH +'bank.csv')
df.head()

## 2. Проведите предварительный разведочный анализ выборки (типы данных, статистики, распределения, визуализация, корреляционный анализ).

In [ ]:
df.info()


In [ ]:
df.describe().T

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x='deposit', y='age', data=df);

In [ ]:
plt.figure(figsize = (14, 6))
sns.boxplot(x='month', y='day', data=df,
            order=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec']);

In [ ]:
cmap = matplotlib.cm.Greens

for value in ['yes', 'no']:
    df_jobs = df[df['deposit'] == value]['job'].value_counts()

    norm = matplotlib.colors.Normalize(vmin=min(df_jobs.values), vmax=max(df_jobs.values))
    colors = [cmap(norm(value)) for value in df_jobs.values]


    fig = plt.gcf()
    fig.set_size_inches(16, 7)
    squarify.plot(sizes=df_jobs.values, label=df_jobs.index, color=colors, alpha=.8 , value=df_jobs.values)
    plt.axis('off')
    plt.show()
    
    cmap = matplotlib.cm.Reds




In [ ]:
fig, axes = plt.subplots(3,2, figsize=(14,14))
columns = ['marital', 'education', 'default', 'housing', 'loan', 'contact']
c = 0

for i in range(len(axes)):
    for j in range(len(axes[i])):
        sns.catplot(y=columns[c], hue="deposit", kind="count",
                    palette="Set2", edgecolor=".5",
                    data=df, ax=axes[i][j])
        plt.close(2)
        c+=1

In [ ]:
 g = sns.pairplot(df, vars=['age', 'balance'], hue="deposit",  palette='Set2')

In [ ]:
 g = sns.pairplot(df, vars=['duration', 'campaign', 'pdays', 'previous'], hue="deposit",  palette='Set2')

## 3. Проведите необходимую предобработку данных (preprocessing). 

Для построения моделей с помощью метрических методов **все признаки должны быть закодированы числами**. Полезными будут следующие методы библиотеки Pandas:
- map() –– для перекодировки категориальной переменной числовыми метками;
- get_dummies() –– для создания нескольких бинарных признаков на основе категориального.

Также может потребоваться масштабирование данных (scaling). Воспользуйтесь классом StandardScaler библиотеки Scikit-learn:
```
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
```
Заметим, что целевую переменную масштабировать, как правило, не требуется.

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
new_values = {'no':0, 'yes':1} 
y = y.map(new_values)

X['loan'] = X['loan'].map(new_values)
X['housing'] = X['housing'].map(new_values)
X['default'] = X['default'].map(new_values)
X.head()

In [ ]:
education_labels = {'unknown':0, 'primary':1, 'secondary':2, 'tertiary':3}
X['education'] = X['education'].map(education_labels)
X.head()

In [ ]:

#X.drop(['contact','day','month','duration','campaign','pdays','previous','poutcome'],axis=1, inplace=True)
for column in ['poutcome', 'contact', 'month']:
    dummies = pd.get_dummies(X[column], drop_first=True, prefix=column)
    X = pd.concat([X, dummies],axis=1)
    X.drop([column],axis=1, inplace=True)
X.head()

In [ ]:
for column in ['job', 'marital']:
    dummies = pd.get_dummies(X[column], drop_first=True, prefix=column)
    X = pd.concat([X, dummies],axis=1)
    X.drop([column],axis=1, inplace=True)
X.head()

In [ ]:
X.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 4. Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split в соотношении 75 на 25.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=19)


## 5. Обучите алгоритм классификации kNeighborsClassifier или регрессии KNeighborsRegressor. 

Оцените качество каждой модели на валидационной выборке с помощью
- **accuracy_score** для классификации;
- **mean_squared_error** для регрессии.
Сравните результаты и сделайте выводы.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

## 6. Создайте генератор разбиений, который перемешивает выборку перед созданием блоков (shuffle=True).

Число блоков **n_splits** равно 5. Задайте также параметр **random_state** для воспроизводимости результатов. Например:
```
kf = KFold(n_splits=5, shuffle=True, random_state=42)
```
Найдите показатель качества модели kNN на кросс-валидации. Подумайте, приемлемо ли использование вашей меры (метрики) качества в данной задаче? При необходимости пересчитайте качество с помощью другой метрики из [списка](https://scikit-learn.org/stable/modules/model_evaluation.html).

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X_scaled, y, 
                         cv=kf, scoring='accuracy')
print(scores)
mean_score = scores.mean()
print(mean_score)

## 7. Осуществите кросс-валидацию модели при числе соседей от 1 до 50.

Используйте **GridSearchCV**. При каком $k$ качество получилось наилучшим? Чему равна
эта оценка качества? Постройте график значений метрики в зависимости от $k$
(matplotlib.pyplot.plot()).

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 50)} # число соседей -- от 1 до 50
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=kf) 
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
knn_grid.best_params_

In [ ]:
results = pd.DataFrame(knn_grid.cv_results_)
results

In [ ]:
y_pred = knn_grid.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=7)
y_pred = best_knn.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
sns.lineplot(x='param_n_neighbors', y='mean_test_score', data=results);